# Using TrackMap on Grafana

<img src="img/by-nc.png" alt="Attribution 4.0 International (CC BY 4.0)" width="100"> [Attribution 4.0 International (CC BY 4.0)](https://creativecommons.org/licenses/by-nc/4.0/) 2021-07-10 by [canne](https://github.com/canne)

We expect that one has Grafana / IndluxDB / nginx enabled on a Docker environment as explained in the manual, chapter [InfluxDB](../../readthedocs/source/influxdb/influxdb.ipynb#Grafana).

## TrackMap installation

Plese refer first to [plugin's pages](https://grafana.com/grafana/plugins/pr0ps-trackmap-panel/?tab=installation).

The installations is done from command line with Grafana CLI in our standalone (local) system. On Windows, the easiest way to get it with with Docker Dashboard, navigate to the `dasht_grafana` container and click on the `CLI` button. Give command:

```
/usr/share/grafana $ grafana-cli plugins install pr0ps-trackmap-panel
installing pr0ps-trackmap-panel @ 2.1.2
from: https://grafana.com/api/plugins/pr0ps-trackmap-panel/versions/2.1.2/download
into: /var/lib/grafana/plugins

* Installed pr0ps-trackmap-panel successfully

Restart grafana after installing plugins . <service grafana-server restart>
```

As suggested, close the CLI windows, head back to the Docker Dashboard and restart the container `dasht_grafana`.

Using the Grafana administrator interface, you can now observe a new, signed plugin, `TrackMap`. When you create a panel, it shows up as an alternative `Visualization` method under the corresponding drop down menu. If it does not, you may need to restart the Grafana server, which can be used simply by first stopping and the starting the entire _DashT_ Docker stack.

## TrackMap and Flux

Apparently, TrackMap is a GIS application and only SQL is used. However, from the Grafana's point of view it is about the query and the fields it returns.

Somebody using TrackMap [has had the issue already this year and come with a solution where the Flux itself has been used to map the fields the query produces](https://github.com/alexandrainst/alexandra-trackmap-panel/issues/47#issuecomment-792658973). He came up with a solution like this:

```
from(bucket: "home_assistant")
  |> range(start: v.timeRangeStart, stop: v.timeRangeStop)
  |> filter(fn: (r) => r["_measurement"] == "device_tracker.mate10")
  |> filter(fn: (r) => r["_field"] == "gps_accuracy" or r["_field"] == "latitude" or r["_field"] == "longitude" or r["_field"] == "velocity")

|> pivot(columnKey: ["_field"], rowKey: ["_time"], valueColumn: "_value")

|> duplicate(column: "latitude", as: "lat")
|> duplicate(column: "longitude", as: "lon")
|> duplicate(column: "_time", as: "tooltip")
|> duplicate(column: "velocity", as: "popup")

|> map(fn: (r) => ({ r with popup: "Speed: " + string(v:r.popup) + " km/h" }))

|> keep(columns: ["_time", "tooltip", "lat", "lon", "popup"])
```

The above creates some errors in the InfluxDB query but provides `lat` and `lon` arrays alright in _DashT_ use case with Grafana.

I rather make queries for all parameters in one single panel, where I can observe the field results, say in a Table visualization. Then, in the TrackMap panel I would read from this panel and use [Flux Transformation functions](https://docs.influxdata.com/influxdb/v2.0/reference/flux/stdlib/built-in/transformations/) I can select which queries are actually executed and give new names to the fields, etc.

[Here is the Grafana 7.5.7 JSON-model](./GrafanaTrackMapTws.json) to be imported as a Dashboard. This the same example dashboard used in making the TrackMap example in the User's Manual, it looked like this in its total size. Note that only one panel is making data inquiries (multiple) from the database, others are using and transforming that inquiry inside Grafana:

<img src="img/s_005_Grafana_TrackMap_Position_and_TWS.png"
alt="Grafana - Dashboard with position" width="700">[(zoom)](img/005_Grafana_TrackMap_Position_and_TWS.png)

### More complex example

The above dashboard with TrackMap and a centralized, single panel making all queries, one per data parameter, which can be distributed to other panels, selection taking place in the Transformations post-processing.

The data is also historical, from 2018, and input from a VDR record file but with sufficient timestamps from GPS allowing the approximation of the timestamps in between the GPS ticks using the CPU clock. With Grafana level display, or even when zooming in with InfluxDB, there is is no discontinuity visible.

[Here is the Grafana 7.x JSON-model](./GrafanTrackMapPerformance.json) of the Dashoard making the panels below. The data is from my personal historical roadbooks. Like in the previous example, there is a single panel hidden below which sole usage is to make all data inquiries from the database while the other, performance oriented databases are requesting the data internally, within Grafana.

<img src="img/s_010_Grafana_TrackMap_Position_and_Performances.png"
alt="Grafana - Performance Analysis Dashboard with position" width="700">[(zoom)](img/010_Grafana_TrackMap_Position_and_Performances.png)

## Alexandra Institute's Track Map

Since we dropped on this track map as well, let's record here the experience: It looks promising with its heat map - one could show the intensity of the wind, or the polar performance!

There is a [zip/tar.gz distribution](https://github.com/alexandrainst/alexandra-trackmap-panel/releases) of this more complete (and more complex) GIS tool.

Somebody using Alexandra Track Map [has had the issue already this year and come with a solution where the Flux itself has been used to map the fields the query produces](https://github.com/alexandrainst/alexandra-trackmap-panel/issues/47#issuecomment-792658973). He came up with a solution with pure Flux filtering (see the link).

I tried it with Alexandra Track Map v2.2.1, some adaptation as from the Data Explorer of InfluxDB copying the first part for the query, but in vain.

It is noteworthy that Alexandra Track Map does v2.2.1 does not deal with the Grafana transform functions but it must be the flat query which needs to provide all columns. This makes it quite complicated with Flux to get them joined: the resulting two tables are well understood by other modules, but not this one.

Others have had this issue as well and provided [a pull request](https://github.com/alexandrainst/alexandra-trackmap-panel/commit/5c2d62aae180aa00df6f68729c61a52738b832c1) to overcome this by providing 'customs' field names for _lat_ and _lon_, see `options.coordinates` in [GrafanaAlexsandraTrackMapCustom.json](./GrafanaAlexsandraTrackMapCustom.json). I installed this version but it would work only with SQL, with Flux there are two tables always underneath and since this plugin cannot deal with _join_ transformation by time, it is quite hopeless since the above _pivot()_ table solution does not work, either.